# Dataformats - XML
We'd like to exchange some data with another partner, the partner uses a NoSQL database provider and we don't know how his application stores the data internally. We agreed to use XML as data format.
> You can only use JSON (which is quite common nowadays) by replacing the XML with JSON statements.

## Transforming our data
For of all we'd like to extract all our `products` to the partner in XML format.

In [8]:
SELECT TOP 3 * 
FROM Product
FOR XML AUTO

## Manipulating our data
To manipulate our data we can just adjust the `SELECT` clause. We only like to share the `name`, `sizes` and `cost`

In [9]:
SELECT TOP 3 
    ProductName AS Name,
    Price AS Price,
    Sizes AS Sizes
FROM Product
FOR XML AUTO

# Joining our data
The partner would also like to know the `type` of the `products`, we can easily join the `ProductType` table.

In [10]:
SELECT TOP 3 
    ProductName AS Name,
    Price AS Price,
    Sizes AS Sizes,
    ProductTypeName AS Type
FROM Product
JOIN ProductType on ProductType.ProductTypeID = Product.ProductTypeID
FOR XML AUTO

## Storing Data
Let's say if we inverted the transaction, we'll be **receiving** XML data instead of sending it. You can easily create a XML data column in SQL Server or use variables to store information. 

In [12]:
DROP TABLE TestXmlTable;

CREATE TABLE TestXmlTable(
    Col1 INT PRIMARY KEY IDENTITY,
    Col2 XML -- see what we did here?
);


To insert data into the table we can create a variable or direct insert a XML string. Manipulating data is about the same so you'll get the idea.

In [18]:
-- Clear the table's data.
DELETE FROM TestXmlTable;

-- Assign a XML like string to a variable
DECLARE @myString VARCHAR(100)   
SET @myString = '<Cust><Fname>Andrew</Fname><Lname>Fuller</Lname></Cust>' 

-- Insert XML Data into the table
INSERT INTO TestXmlTable(Col2)
VALUES(@myString)

SELECT * FROM TestXmlTable

# Bulk Insert Data
To insert data by bulk we can use the `OPENROWSET()` statement
The following example shows how to insert a row in table `T1`. The value of the XML column is loaded from file `data\xmlsample.xml` as BLOB.
> It's required to have the file downloaded at the `C:\temp\data` location or change the relative path if needed.

Contents of the xmlsample.xml
```xml
<?xml version="1.0" encoding="UTF-8"?>  
<Root>  
    <ProductDescription ProductModelID="5">  
        <Summary>Some Text</Summary>  
    </ProductDescription>  
</Root> 
```

In [5]:
-- Clear the table's data.
DELETE FROM TestXmlTable;

-- Insert
INSERT INTO TestXmlTable(Col2)
SELECT * FROM OPENROWSET(  
   BULK 'C:\temp\data\xmlsample.xml',  
   SINGLE_BLOB) as Col2;  

-- Read the data.
SELECT * FROM TestXmlTable

# Reading XML columns with `XQuery`
Let's state that we only want to see the `<Summary>` of a product, we can do so using `XQuery`.

In [11]:
SELECT Col2.query('/Root/ProductDescription/Summary') 
FROM TestXmlTable

(1 row affected)

Total execution time: 00:00:00.0059307

(No column name)
<Summary>Some Text</Summary>
